<a href="https://colab.research.google.com/github/daniivelascoo/ifp-programacion-ia/blob/main/Teoria_1_3_B_Limpieza_Transformacion_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Teoría Bloque B: Limpieza y Transformación
**Sprint 1.3 - Pandas Deep Dive**

Los datos reales están sucios. Tienen huecos vacíos (`NaN`), duplicados y formatos extraños.
Antes de hacer IA, tenemos que hacer de "fontaneros" de datos.

### 🎯 Objetivos:
1.  **Nulos:** Detectar y rellenar huecos vacíos (`fillna`).
2.  **Duplicados:** Eliminar repeticiones (`drop_duplicates`).
3.  **Ingeniería de Características:** Crear nuevas columnas a partir de las existentes.
4.  **El poder de `.apply()`:** Usar tus propias funciones en Pandas.

## 1. Tratamiento de Valores Nulos (NaN)

En Python, un dato faltante se representa como `NaN` (Not a Number) o `None`.
La mayoría de modelos de IA fallan si encuentran un `NaN`.

Tenemos dos estrategias:
1.  **Borrar:** Si hay pocos nulos, eliminamos la fila.
2.  **Imputar:** Rellenamos el hueco con lógica (Media, 0, "Desconocido").

In [1]:
import pandas as pd
import numpy as np

# Creamos un DataFrame sucio
df = pd.DataFrame({
    'producto': ['Manzana', 'Pera', 'Plátano', 'Manzana', 'Uva'],
    'precio': [100, np.nan, 50, 100, 200], # Un precio desconocido
    'stock': [10, 5, 20, 10, np.nan]       # Un stock desconocido
})

print("--- DataFrame Sucio ---")
display(df)

# 1. DETECTAR NULOS
# .isnull() devuelve True si hay hueco. .sum() los cuenta.
conteo_nulos = df.isnull().sum() # TODO: Suma los nulos
print(f"\nRecuento de nulos:\n{conteo_nulos}")

# 2. ESTRATEGIA A: BORRAR (Drástica)
# Eliminamos cualquier fila que tenga al menos un fallo
df_borrado = df.dropna()
print("\n--- Después de dropna (Perdemos datos) ---")
display(df_borrado)

# 3. ESTRATEGIA B: RELLENAR (Inteligente)
# Vamos a rellenar el precio vacío con la MEDIA de los precios.
media_precio = df['precio'].mean()
print(f"\nPrecio medio calculado: {media_precio}")

# Rellenamos solo la columna precio
# IMPORTANTE: Hay que guardar el resultado en la columna de nuevo
df['precio'] = df['precio'].fillna(media_precio) # TODO: Pon la variable media_precio

print("\n--- Precio arreglado ---")
display(df)

--- DataFrame Sucio ---


,producto,precio,stock
0,Manzana,100.0,10.0
1,Pera,NaN,5.0
2,Plátano,50.0,20.0
3,Manzana,100.0,10.0
4,Uva,200.0,NaN



Recuento de nulos:
producto    0
precio      1
stock       1
dtype: int64

--- Después de dropna (Perdemos datos) ---


,producto,precio,stock
0,Manzana,100.0,10.0
2,Plátano,50.0,20.0
3,Manzana,100.0,10.0



Precio medio calculado: 112.5

--- Precio arreglado ---


,producto,precio,stock
0,Manzana,100.0,10.0
1,Pera,112.5,5.0
2,Plátano,50.0,20.0
3,Manzana,100.0,10.0
4,Uva,200.0,NaN


## 2. Eliminación de Duplicados

A veces, por errores de sistema, la misma fila aparece dos veces. Esto sesga las estadísticas.

In [2]:
# Fíjate en el DataFrame original de arriba: 'Manzana' (precio 100, stock 10) sale dos veces.

# 1. VERIFICAR DUPLICADOS
duplicados = df.duplicated().sum()
print(f"Filas totalmente repetidas: {duplicados}")

# 2. ELIMINAR
# keep='first' mantiene la primera aparición y borra las siguientes.
df_limpio = df.drop_duplicates(keep='first')

print("\n--- Sin Duplicados ---")
display(df_limpio)

# Guardamos en 'df' para seguir trabajando
df = df_limpio.copy()

Filas totalmente repetidas: 1

--- Sin Duplicados ---


,producto,precio,stock
0,Manzana,100.0,10.0
1,Pera,112.5,5.0
2,Plátano,50.0,20.0
4,Uva,200.0,NaN


## 3. Crear Columnas (Ingeniería de Características)

A menudo, la información útil no viene en el Excel, hay que calcularla.
*   **Operaciones Vectoriales:** Matemáticas directas (Rápido).
*   **Apply:** Aplicar una función compleja fila a fila (Potente pero lento).

In [3]:
# 1. VECTORIZACIÓN (Lo preferido)
# Queremos calcular el valor total del inventario (Precio * Stock).
# Pandas multiplica columna por columna automáticamente.

# Como aún tenemos un NaN en stock, rellenémoslo con 0 primero
df['stock'] = df['stock'].fillna(0)

df['valor_total'] = df['precio'] * df['stock'] # TODO: Multiplica por la columna df['stock']

print("--- Columna Calculada (Vectorial) ---")
display(df)

# 2. APPLY (La navaja suiza)
# Queremos clasificar los productos:
# - Si precio > 100 -> "Premium"
# - Si no -> "Estándar"

# Definimos una función lógica (o usamos una lambda)
def clasificar(precio):
    if precio > 100:
        return "Premium"
    else:
        return "Estándar"

# Aplicamos la función a la columna precio
df['categoria'] = df['precio'].apply(clasificar) # TODO: Usa el método apply

print("\n--- Columna con Apply ---")
display(df)

--- Columna Calculada (Vectorial) ---


,producto,precio,stock,valor_total
0,Manzana,100.0,10.0,1000.0
1,Pera,112.5,5.0,562.5
2,Plátano,50.0,20.0,1000.0
4,Uva,200.0,0.0,0.0



--- Columna con Apply ---


,producto,precio,stock,valor_total,categoria
0,Manzana,100.0,10.0,1000.0,Estándar
1,Pera,112.5,5.0,562.5,Premium
2,Plátano,50.0,20.0,1000.0,Estándar
4,Uva,200.0,0.0,0.0,Premium
